In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image
import pandas as pd
import cv2


input_dim = 5  # one-hot size
hidden_size = 4  # output from the LSTM. 5 to directly predict one-hot
batch_size = 50   # one sentence
sequence_length = 5  # |ihello| == 6
learning_rate = 0.0001
data_mean = 17.119790415671556
data_std = 63.36037302373175

In [2]:
X = tf.placeholder(tf.float32, [None, 200, 400, 1])
Y = tf.placeholder(tf.float32, [None, 4])

with tf.variable_scope("cnn"):
    
    # conv : 5, 200, 400, 16
    conv = tf.layers.conv2d(inputs=X, filters=16, kernel_size=[4, 8], padding='SAME', activation=tf.nn.relu)
    # pool : 5, 50, 99, 16
    pool = tf.layers.max_pooling2d(inputs=conv, pool_size=[2, 4], padding='SAME', strides=1)
    # conv2 
    conv2 = tf.layers.conv2d(inputs=pool, filters=64, kernel_size=[4, 8], padding='SAME', activation=tf.nn.relu)
    # pool2 
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[4, 8], padding='SAME', strides=2)
    # conv3
    conv3 = tf.layers.conv2d(inputs=pool2, filters=128, kernel_size=[4, 8], padding='SAME', activation=tf.nn.relu)
    # pool3 
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[4, 8], padding='SAME', strides=2)
    
    flat = tf.reshape(pool3, [-1, 50*100*128])
    w = tf.get_variable("w", [hidden_size, hidden_size])
    b = tf.get_variable("b", [hidden_size])
    
    logit = tf.layers.dense(inputs=flat, units=4)
    outputs = tf.sigmoid(tf.matmul(logit, w) + b)
    
    loss = -tf.reduce_mean(Y*tf.log(outputs)+(1-Y)*(tf.log(1-outputs)))
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [ ]:
saver = tf.train.Saver()

with tf.Session() as sess:
    res = raw_input("Do you want addtional learning? (Name or Enter)")
    if res:
        saver.restore(sess, "./model/"+res+".ckpt")
    else:
        sess.run(tf.global_variables_initializer())
    model_name = raw_input("Model_Name: ")
    
    try:
        for epoch in range(10):
            for ims in range(1, 9):

                data_df = pd.read_csv('/mnt/d/imgs/img'+str(1)+'/log.csv', names=['name','key_out'])
                names = data_df['name'].values
                # keyouts = [8, 1, 4, 2, 15, ...]
                key_out = [eval(x) for x in data_df['key_out'].values]

                while len(names):
                    imgs = []
                    offset = min(batch_size, len(names))
                    batch_x = names[0:offset]
                    batch_y = key_out[0:offset]
                    names = names[offset:]
                    key_out = key_out[offset:]

                    for name in batch_x:
                        img = Image.open('/mnt/d/imgs/img'+str(1)+'/img/'+name)
                        data = np.array( img, dtype='uint8')
                        data = data.reshape([200,400,1])
                        data = (data-data_mean) / data_std
                        imgs.append(data)
                    sess.run(train, feed_dict={X:np.array(imgs, np.float32), Y:batch_y})
                    print sess.run(loss, feed_dict={X:np.array(imgs, np.float32), Y:batch_y})
            if epoch == 0:
                save_path = saver.save(sess, "./model/"+model_name+"_1.ckpt")
            if epoch == 4:
                save_path = saver.save(sess, "./model/"+model_name+"_5.ckpt")
            print 'epoch', epoch+1, sess.run(loss, feed_dict={X:np.array(imgs, np.float32), Y:batch_y})
    finally:
        save_path = saver.save(sess, "./model/"+model_name+"_10.ckpt")

Do you want addtional learning? (Name or Enter)
Model_Name: full2
0.35640317
1.5535754
1.2357409
0.73633283
0.64296013
0.4853046
0.50086814
0.4953048
0.34189862
0.6112249
0.541324
0.50452
0.47214836
0.49195588
0.5305465
0.6229396
0.3495359
0.43707588
0.49932626
0.47042054
0.4127766
0.40463936
0.4721344
0.5729366
0.13838959
0.48902434
0.5138089
0.4491828
0.4079789
0.41736594
0.44608584
0.5286056
0.2075904
0.41305178
0.46700698
0.43554443
0.40808892
0.39089888
0.41847992
0.4671894
0.14866851
0.41336647
0.4593477
0.4184109
0.3937027
0.38255635
0.38930976
0.42687613
0.1268781
0.39943847
0.45437372
0.39383224
0.38608468
0.34477356
0.36540335
0.3951006
0.12533699
0.36231187
0.4110781
0.3618203
0.38092405
0.31176692
0.34584287
0.35334757
epoch 1 0.35334757
0.08968952
0.3490272
0.389347
0.33788916
0.3671578
0.30688894
0.3085936
0.31961498
0.09504448
0.33064705
0.36587596
0.31122115
0.34756902
0.31330466
0.30782962
0.37309498
0.14684391
0.32113442
0.36528262
0.31489763
0.33238074
0.25629327
0.2